## Table of Contents

- [Example - San Diego Meshing](#Example---San-Diego-Meshing)
- [Plot Initial Geometry](#Plot-Initial-Geometry)
- [Generate the Mesh](#Generate-the-Mesh)
- [Plot Mesh](#Plot-Mesh)

## Example - San Diego Meshing

This example illustrates how run a more complicated meshing example for a real-world case.  In this case, you use a polygon and other features defining the extents of San Diego Bay to generate a mesh that can be used for running a hydraulic model of the bay. An overview of the model is shown in the image below.

A python-based class has been created for this example that contains two PolyInput objects that can be used to generate a 2D mesh.

In [1]:
from shapely.geometry import Polygon

import meshing_tools
import xms.mesher as mesh

## Plot Initial Geometry

For this example we'll use the polygon data held in the meshing_tools.py file. The code below retreives this polygon data.

In [2]:
san_diego_example = meshing_tools.SanDiegoExample()
poly1 = san_diego_example.poly1
poly2 = san_diego_example.poly2

The code shown below plots our polygon data using geoviews. The plot of this data is shown in the following image.

```python
basemap = gv.tile_sources.StamenTerrainRetina().options(width=900, height=900)

polygons = gv.Polygons(
    data=[
        meshing_tools.polys_as_shapely(poly2), 
        meshing_tools.polys_as_shapely(poly1)
    ], 
    crs=ccrs.UTM(zone=11),
).options(fill_color=None,)

points = gv.Points(
    data=[poly1.outside_poly, poly2.outside_poly, poly2.inside_polys[-1]],
    crs=ccrs.UTM(zone=11),
).options(color='black', size=4)

path = gv.Path(
    data=[pd.DataFrame(data=poly2.inside_polys[-1], columns=['x', 'y', 'z'])], 
    crs=ccrs.UTM(zone=11), 
).options(line_color='black')

basemap * polygons * points * path
```

![Initial Geometry](images/initial_geometry.png)

The MultiPolyMesherIo class holds all the options for generating meshes from polygon data

In [3]:
mesh_io = mesh.meshing.MultiPolyMesherIo((poly1, poly2))

## Generate the Mesh

In [4]:
# Generate Mesh
succeded, errors = mesh.meshing.mesh_utils.generate_mesh(mesh_io=mesh_io)
if succeded:
    print("Meshing was successful")
else:
    print("Meshing errors found:")
    for err in errors:
        print("\t{}".format(err))

Meshing was successful


## Plot Mesh
The mesh is converted to a pandas dataframe which is then used by geoviews for plotting. The image below shows the plot generated by the geoviews code.

```python
verts,cells = meshing_tools.xmsmesh_to_dataframe(mesh_io.points, mesh_io.cells)
vert_points = gv.project(gv.Points(verts, vdims=['z'], crs=ccrs.UTM(zone=11)))
trimesh = gv.TriMesh((cells, vert_points))

basemap * polygons * points * path * trimesh.edgepaths.options(line_width=1)
```

![The generated mesh](images/mesh.png)